In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:
f = open('D:/Photo/index.html', 'r')

soup = BeautifulSoup(f, 'lxml')
reports = soup.findAll('a', class_="item item--reportage")

data = []
workers = []

for report in reports:
    url = report.get('href')
    date = report.find('div', class_="item__desc-top__date").text
    top_title = report.find('div', class_="item__desc-top__title").text
    title = report.find('div', class_="item__desc-title").text

    data.append([url, date, top_title, title])

for report in data:
    r = requests.get(report[0])
    soup = BeautifulSoup(r.text, 'lxml')
    a = soup.findAll('div', class_="titles__title")
    url = "https://geo.pro" + a[1].find('a').get('href')

    workers.append(url)

In [3]:
df = pd.DataFrame(workers)
df = df.drop_duplicates()
df.to_excel("D:\Photo\output_workers.xlsx")

In [4]:
arr = df.to_numpy()

In [59]:
data = []

for worker in arr:
    #Получение кол-ва страниц paginator'а
    last_page = 0
    r = requests.get(worker[0])
    soup = BeautifulSoup(r.text, 'lxml')
    pl = soup.findAll('a', class_="paginator__link")
    for p in pl:
        last_page = int(p.text)
    print(last_page)

    for p in range(1, last_page):
        #Получение ссылок репортажей со страницы фотографа
        if last_page < 5:
            url = f"{worker[0]}?PAGEN_1={p}"
        else:
            url = f"{worker[0]}?PAGEN_2={p}"
        r = requests.get(url)
        print(url)            
        #sleep(3)
        soup = BeautifulSoup(r.text, 'lxml')
        reports = soup.findAll('a', class_="item")

        for report in reports:
            url = "https://geo.pro" + report.get('href')
            r = requests.get(url)
            soup = BeautifulSoup(r.text, 'lxml')
            subtitle = soup.find('div', class_="titles__subtitle").text
            title = soup.find('div', class_="titles__title").text
            date = soup.find('div', class_="titles__date").text
            views = soup.find('span', id="countViewReportage").text
            photo = soup.find('span', id="countPhotoReportage").text

            print(url, subtitle, title, date, views, photo, worker[0])
            data.append([url, subtitle, title, date, views, photo, worker[0]])

df = pd.DataFrame(data)
df.to_excel("D:\Photo\output.xlsx")   

5
https://geo.pro/worker/vladislav-zoomsky/?PAGEN_2=1
https://geo.pro/reportage/700535-mon-etoile-startovyy-seminar/ 
События  
                Mon Etoile. Стартовый семинар             
                                5 февраля             <span id="countViewReportage"> </span> 139 https://geo.pro/worker/vladislav-zoomsky/
https://geo.pro/reportage/698644-novogodnyaya-noch-v-restorane-kakadu/ 
KAKADU  
                Новогодняя ночь в ресторане KAKADU             
                                1 января             <span id="countViewReportage"> </span> 264 https://geo.pro/worker/vladislav-zoomsky/
https://geo.pro/reportage/698191-ohota-26-12/ 
Охота на зайцев  
                #OHOTA 26/12             
                                26.12.2021             <span id="countViewReportage"> </span> 86 https://geo.pro/worker/vladislav-zoomsky/
https://geo.pro/reportage/698169-shtalman/ 
Ресторан Штальман  
                #ШТАЛЬМАН             
                                26.12.2021

KeyboardInterrupt: 